In [1]:
# Step 1: Install Required Dependencies
!pip install -q langchain langchain-community langchain-google-genai requests duckduckgo-search python-dotenv ipywidgets
print("✅ All dependencies installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18

In [2]:
# Step 2: Import All Required Libraries
import os
import requests
import json
from typing import Optional, Type
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import Layout, Button, VBox, HBox, Output, Text, HTML as HTMLWidget

# LangChain imports
from langchain.tools import BaseTool, tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_google_genai import ChatGoogleGenerativeAI

# Google Colab specific
from google.colab import userdata

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [3]:
# Step 3: Configure API Keys from Colab Secrets
def setup_api_keys():
    """Setup and verify API keys from Colab secrets"""
    global GOOGLE_API_KEY, WEATHER_API_KEY

    try:
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
        WEATHER_API_KEY = userdata.get('WEATHER_API_KEY')

        # Display status with beautiful formatting
        display(HTML("""
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    padding: 20px; border-radius: 15px; color: white; margin: 10px 0;">
            <h3 style="margin: 0 0 15px 0;">🔑 API Keys Configuration</h3>
        """))

        if GOOGLE_API_KEY:
            display(HTML('<p style="color: #4CAF50; font-weight: bold;">✅ Google API Key: Configured</p>'))
        else:
            display(HTML('<p style="color: #f44336; font-weight: bold;">❌ Google API Key: Missing</p>'))

        if WEATHER_API_KEY:
            display(HTML('<p style="color: #4CAF50; font-weight: bold;">✅ Weather API Key: Configured</p>'))
        else:
            display(HTML('<p style="color: #f44336; font-weight: bold;">❌ Weather API Key: Missing</p>'))

        display(HTML('</div>'))

        return bool(GOOGLE_API_KEY and WEATHER_API_KEY)

    except Exception as e:
        display(HTML(f'<div style="color: red; padding: 10px;">❌ Error configuring API keys: {str(e)}</div>'))
        return False

# Run the setup
api_keys_configured = setup_api_keys()

if not api_keys_configured:
    display(HTML("""
    <div style="background: #fff3cd; border: 1px solid #ffeaa7;
                padding: 15px; border-radius: 10px; margin: 10px 0;">
        <h4 style="color: #856404; margin: 0 0 10px 0;">⚠️ Setup Required</h4>
        <p style="color: #856404; margin: 0;">
            Please add your API keys to Colab secrets:
            <br>• Click the 🔑 icon in the left sidebar
            <br>• Add GOOGLE_API_KEY (from Google AI Studio)
            <br>• Add WEATHER_API_KEY (from WeatherAPI.com)
        </p>
    </div>
    """))

In [4]:
# Step 4: Initialize Google Gemini LLM
def initialize_llm():
    """Initialize and test the Gemini LLM"""
    global llm

    if not GOOGLE_API_KEY:
        display(HTML('<div style="color: red; padding: 10px;">❌ Cannot initialize LLM: Google API key missing</div>'))
        return False

    try:
        # Initialize Gemini 1.5 Flash
        llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0.1,
            google_api_key=GOOGLE_API_KEY,
            convert_system_message_to_human=True
        )

        # Test the connection
        test_response = llm.invoke("Hello! Please respond with just 'LLM working perfectly!'")

        display(HTML("""
        <div style="background: linear-gradient(135deg, #4CAF50 0%, #45a049 100%);
                    padding: 20px; border-radius: 15px; color: white; margin: 10px 0;">
            <h3 style="margin: 0 0 10px 0;">🤖 Gemini 1.5 Flash Initialized</h3>
            <p style="margin: 0; opacity: 0.9;">Model: gemini-1.5-flash (Free Tier)</p>
            <p style="margin: 5px 0 0 0; opacity: 0.9;">Status: ✅ Connected and Ready</p>
        </div>
        """))

        return True

    except Exception as e:
        display(HTML(f"""
        <div style="background: #f44336; padding: 15px; border-radius: 10px; color: white; margin: 10px 0;">
            <h4 style="margin: 0 0 10px 0;">❌ LLM Initialization Failed</h4>
            <p style="margin: 0; opacity: 0.9;">Error: {str(e)}</p>
        </div>
        """))
        return False

# Initialize the LLM
llm_initialized = initialize_llm()

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [5]:
# Step 5: Create Optimized Weather Tool
@tool
def get_weather_forecast(city: str) -> str:
    """
    Fetch current weather and 3-day forecast for a city using WeatherAPI.

    Args:
        city (str): City name to get weather for

    Returns:
        str: Formatted weather information
    """
    if not WEATHER_API_KEY:
        return "❌ Weather API key not configured. Please add WEATHER_API_KEY to Colab secrets."

    try:
        url = "http://api.weatherapi.com/v1/forecast.json"
        params = {
            'key': WEATHER_API_KEY,
            'q': city,
            'days': 3,
            'aqi': 'no',
            'alerts': 'no'
        }

        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        # Extract data
        current = data['current']
        location = data['location']
        forecast = data['forecast']['forecastday']

        # Format weather info
        weather_info = f"""🌍 **{location['name']}, {location['country']}**

📍 **Current Weather:**
• Temperature: {current['temp_c']}°C ({current['temp_f']}°F)
• Condition: {current['condition']['text']}
• Feels like: {current['feelslike_c']}°C
• Humidity: {current['humidity']}%
• Wind: {current['wind_kph']} km/h {current['wind_dir']}

📅 **3-Day Forecast:**"""

        for day in forecast:
            date_parts = day['date'].split('-')
            formatted_date = f"{date_parts[2]}/{date_parts[1]}"
            day_info = day['day']
            weather_info += f"""
• {formatted_date}: {day_info['condition']['text']}
  High: {day_info['maxtemp_c']}°C | Low: {day_info['mintemp_c']}°C | Rain: {day_info['daily_chance_of_rain']}%"""

        return weather_info

    except requests.exceptions.Timeout:
        return f"❌ Weather request timed out for {city}. Please try again."
    except requests.exceptions.RequestException as e:
        return f"❌ Network error getting weather for {city}: {str(e)}"
    except KeyError as e:
        return f"❌ Weather data parsing error for {city}: Missing {str(e)}"
    except Exception as e:
        return f"❌ Unexpected error getting weather for {city}: {str(e)}"

# Test the weather tool
display(HTML("""
<div style="background: linear-gradient(135deg, #FF6B6B 0%, #4ECDC4 100%);
            padding: 15px; border-radius: 10px; color: white; margin: 10px 0;">
    <h4 style="margin: 0 0 10px 0;">🌤️ Weather Tool Created</h4>
    <p style="margin: 0; opacity: 0.9;">Testing with London...</p>
</div>
"""))

test_weather = get_weather_forecast("London")
print("Sample weather output:")
print(test_weather[:200] + "..." if len(test_weather) > 200 else test_weather)

<ipython-input-5-1793592391>:75: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  test_weather = get_weather_forecast("London")


Sample weather output:
🌍 **London, United Kingdom**

📍 **Current Weather:**
• Temperature: 21.2°C (70.2°F)
• Condition: Sunny
• Feels like: 21.2°C
• Humidity: 56%
• Wind: 5.4 km/h W

📅 **3-Day Forecast:**
• 16/06: Sunny
  H...


In [6]:
# Step 6: Create Optimized Tourist Attractions Tool
# Initialize search tool
search_tool = DuckDuckGoSearchRun()

@tool
def get_tourist_attractions(city: str) -> str:
    """
    Search for top tourist attractions in a city using DuckDuckGo.

    Args:
        city (str): City name to search attractions for

    Returns:
        str: Formatted tourist attractions information
    """
    try:
        # Optimized search query
        search_query = f"top 10 tourist attractions {city} must visit places travel guide 2024"

        # Perform search with error handling
        search_results = search_tool.run(search_query)

        # Clean and format results
        if not search_results or len(search_results.strip()) < 50:
            return f"❌ Limited information found for tourist attractions in {city}. Try searching online for '{city} tourist attractions'."

        # Format the response
        formatted_results = f"""🏛️ **Top Tourist Attractions in {city}**

{search_results}

💡 **Travel Tips for {city}:**
• Check attraction opening hours and book tickets in advance
• Consider city passes for multiple attractions
• Use public transport or walk when possible
• Try local cuisine and visit traditional markets
• Respect local customs and dress codes
• Take photos but be mindful of photography restrictions"""

        return formatted_results

    except Exception as e:
        return f"❌ Error searching attractions for {city}: {str(e)}. Please try searching manually for '{city} tourist attractions'."

# Test the attractions tool
display(HTML("""
<div style="background: linear-gradient(135deg, #A8E6CF 0%, #88D8C0 100%);
            padding: 15px; border-radius: 10px; color: white; margin: 10px 0;">
    <h4 style="margin: 0 0 10px 0;">🏛️ Tourist Attractions Tool Created</h4>
    <p style="margin: 0; opacity: 0.9;">Testing with Paris...</p>
</div>
"""))

test_attractions = get_tourist_attractions("Paris")
print("Sample attractions output:")
print(test_attractions[:300] + "..." if len(test_attractions) > 300 else test_attractions)

Sample attractions output:
🏛️ **Top Tourist Attractions in Paris**

10 Top Paris Attractions Whether you visit Paris for the winter holiday markets or the summer pop-up beaches, here are the must-see spots, shops and sips in the City of Light. Forbes Travel Guide's list of Paris' must-see attractions, shops and places to eat ...


In [7]:
# Step 7: Create and Configure Travel Agent
def setup_travel_agent():
    """Setup the travel assistant agent with tools"""
    global agent_executor

    if not llm_initialized:
        display(HTML('<div style="color: red; padding: 10px;">❌ Cannot create agent: LLM not initialized</div>'))
        return False

    try:
        # Define tools
        tools = [get_weather_forecast, get_tourist_attractions]

        # Create optimized prompt
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an expert Travel Assistant AI specializing in travel planning and destination information.

🎯 **Your Capabilities:**
• Provide current weather conditions and forecasts
• Recommend top tourist attractions and activities
• Offer practical travel advice and tips
• Help with trip planning and itinerary suggestions

📋 **Instructions:**
• Always be enthusiastic, helpful, and informative
• Use tools to get real-time weather and attraction data
• Format responses clearly with emojis and proper sections
• Provide practical travel tips when relevant
• Be encouraging about travel experiences
• If asked about multiple cities, handle each separately
• Keep responses comprehensive but not overwhelming

🛠️ **Available Tools:**
• get_weather_forecast: Real-time weather data and 3-day forecasts
• get_tourist_attractions: Top attractions and places to visit

Format your responses professionally with clear sections and helpful emojis."""),

            ("user", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ])

        # Create agent
        agent = create_tool_calling_agent(llm, tools, prompt)

        # Create executor with optimized settings
        agent_executor = AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=False,  # Clean output
            handle_parsing_errors=True,
            max_iterations=3,  # Prevent long processing
            return_intermediate_steps=False
        )

        display(HTML("""
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    padding: 20px; border-radius: 15px; color: white; margin: 10px 0;">
            <h3 style="margin: 0 0 15px 0;">🤖 Travel Assistant Agent Ready!</h3>
            <div style="display: flex; justify-content: space-between; flex-wrap: wrap;">
                <div style="flex: 1; margin-right: 20px;">
                    <h4 style="margin: 0 0 10px 0;">✅ Features Enabled:</h4>
                    <ul style="margin: 0; padding-left: 20px;">
                        <li>Real-time weather data</li>
                        <li>Tourist attractions search</li>
                        <li>Travel recommendations</li>
                        <li>Multi-city support</li>
                    </ul>
                </div>
                <div style="flex: 1;">
                    <h4 style="margin: 0 0 10px 0;">🔧 Agent Configuration:</h4>
                    <ul style="margin: 0; padding-left: 20px;">
                        <li>Model: Gemini 1.5 Flash</li>
                        <li>Max iterations: 3</li>
                        <li>Error handling: Enabled</li>
                        <li>Tools: Weather + Attractions</li>
                    </ul>
                </div>
            </div>
        </div>
        """))

        return True

    except Exception as e:
        display(HTML(f"""
        <div style="background: #f44336; padding: 15px; border-radius: 10px; color: white; margin: 10px 0;">
            <h4 style="margin: 0 0 10px 0;">❌ Agent Setup Failed</h4>
            <p style="margin: 0;">Error: {str(e)}</p>
        </div>
        """))
        return False

# Setup the agent
agent_ready = setup_travel_agent()

In [8]:
# Step 8: Create Beautiful Interactive UI
class TravelAssistantUI:
    def __init__(self):
        self.setup_ui()

    def setup_ui(self):
        """Create the interactive UI components"""

        # Custom CSS for beautiful styling
        self.css = """
        <style>
        .travel-container {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 30px;
            border-radius: 20px;
            color: white;
            margin: 20px 0;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
        }
        .travel-title {
            font-size: 2.5em;
            text-align: center;
            margin: 0 0 20px 0;
            text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
        }
        .travel-subtitle {
            text-align: center;
            font-size: 1.2em;
            opacity: 0.9;
            margin-bottom: 30px;
        }
        .response-container {
            background: white;
            color: #333;
            padding: 25px;
            border-radius: 15px;
            margin: 15px 0;
            box-shadow: 0 4px 15px rgba(0,0,0,0.1);
            border-left: 5px solid #667eea;
        }
        .loading-container {
            background: linear-gradient(135deg, #4ECDC4 0%, #44A08D 100%);
            color: white;
            padding: 20px;
            border-radius: 15px;
            text-align: center;
            margin: 15px 0;
        }
        .error-container {
            background: linear-gradient(135deg, #FF6B6B 0%, #FF8E53 100%);
            color: white;
            padding: 20px;
            border-radius: 15px;
            margin: 15px 0;
        }
        </style>
        """

        # Header
        self.header = HTMLWidget(self.css + """
        <div class="travel-container">
            <h1 class="travel-title">🌍 Intelligent Travel Assistant</h1>
            <p class="travel-subtitle">
                Get real-time weather forecasts and discover amazing tourist attractions for any destination worldwide!
            </p>
        </div>
        """)

        # Input components
        self.city_input = Text(
            placeholder="Enter city name (e.g., Paris, Tokyo, New York)",
            description="🏙️ Destination:",
            style={'description_width': 'initial'},
            layout=Layout(width='100%', height='50px')
        )

        self.search_button = Button(
            description="🔍 Explore Destination",
            button_style='primary',
            layout=Layout(width='200px', height='50px'),
            style={'font_weight': 'bold'}
        )

        self.clear_button = Button(
            description="🗑️ Clear Results",
            button_style='warning',
            layout=Layout(width='150px', height='50px')
        )

        # Output area
        self.output_area = Output()

        # Event handlers
        self.search_button.on_click(self.on_search_click)
        self.clear_button.on_click(self.on_clear_click)
        self.city_input.on_submit(self.on_search_submit)

        # Layout
        self.input_box = HBox([
            self.city_input,
            self.search_button,
            self.clear_button
        ], layout=Layout(margin='10px 0'))

        self.ui = VBox([
            self.header,
            self.input_box,
            self.output_area
        ])

    def display_loading(self, city):
        """Show loading animation"""
        with self.output_area:
            display(HTML(f"""
            <div class="loading-container">
                <h3>🔍 Exploring {city}...</h3>
                <p>Gathering weather data and tourist attractions</p>
                <div style="font-size: 2em; animation: pulse 2s infinite;">
                    🌍 ✈️ 🏛️ 🌤️
                </div>
            </div>
            <style>
            @keyframes pulse {{
                0% {{ opacity: 0.5; }}
                50% {{ opacity: 1; }}
                100% {{ opacity: 0.5; }}
            }}
            </style>
            """))

    def display_response(self, response):
        """Display the travel assistant response"""
        with self.output_area:
            display(HTML(f"""
            <div class="response-container">
                <div style="white-space: pre-wrap; line-height: 1.6; font-size: 1.1em;">
{response}
                </div>
            </div>
            """))

    def display_error(self, error_msg):
        """Display error message"""
        with self.output_area:
            display(HTML(f"""
            <div class="error-container">
                <h3>❌ Oops! Something went wrong</h3>
                <p>{error_msg}</p>
                <p><strong>💡 Try:</strong> Check your spelling or try a different city name</p>
            </div>
            """))

    def on_search_click(self, button):
        """Handle search button click"""
        self.search_destination()

    def on_search_submit(self, text_widget):
        """Handle enter key press in text input"""
        self.search_destination()

    def on_clear_click(self, button):
        """Clear all results"""
        self.output_area.clear_output()

    def search_destination(self):
        """Search for destination information"""
        city = self.city_input.value.strip()

        if not city:
            self.display_error("Please enter a city name!")
            return

        if not agent_ready:
            self.display_error("Travel agent is not ready. Please check your API keys and try again.")
            return

        # Clear previous results and show loading
        self.output_area.clear_output()
        self.display_loading(city)

        try:
            # Create search query
            query = f"I want to travel to {city}. Please provide me with current weather information and top tourist attractions for this destination."

            # Get response from agent
            result = agent_executor.invoke({"input": query})

            # Clear loading and display result
            self.output_area.clear_output()
            self.display_response(result['output'])

        except Exception as e:
            self.output_area.clear_output()
            self.display_error(f"Error getting travel information: {str(e)}")

    def show(self):
        """Display the UI"""
        display(self.ui)

# Create and display the UI
if agent_ready:
    travel_ui = TravelAssistantUI()
    travel_ui.show()

    display(HTML("""
    <div style="background: #e8f5e8; border: 1px solid #4CAF50;
                padding: 15px; border-radius: 10px; margin: 20px 0;">
        <h4 style="color: #2e7d32; margin: 0 0 10px 0;">🎉 Travel Assistant Ready!</h4>
        <p style="color: #2e7d32; margin: 0;">
            Enter any city name above and click "Explore Destination" to get weather forecasts and tourist attractions!
        </p>
    </div>
    """))
else:
    display(HTML("""
    <div style="background: #ffebee; border: 1px solid #f44336;
                padding: 15px; border-radius: 10px; margin: 20px 0;">
        <h4 style="color: #c62828; margin: 0 0 10px 0;">⚠️ Setup Required</h4>
        <p style="color: #c62828; margin: 0;">
            Please ensure all API keys are configured and run all previous steps before using the UI.
        </p>
    </div>
    """))

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_hu